# Loop data, Part 3: Exploring CGM data (daily stats, etc)

Before I put the CGM and Loop datasets together, I want to first just explore the CGM dataset, and think about the types of statistics and summary information I want to pull out of the CGM set. I also have some previous code that I wrote to do really basic visualizations in terms of time in range per day, for example. 

In [57]:
import sys
sys.path.append("../")

from mdb_tools import load_data as ld
from mdb_tools import schemas
from mdb_tools import loop_stats as oop
from mdb_tools import sugar_plots as sp

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
yml_secrets_file = '../../secrets/mdb_secrets.yml'

# Access the database using the yml secrets file, and get a specific set of "collections"
col_entries, col_treatments, col_profile, col_devicestatus = ld.get_collections(yml_secrets_file)

# Grab schemas
entries_schema, treatments_schema, devicestatus_schema = schemas.mdb_schemas()

# Load
df_entries = col_entries.find_pandas_all({}, schema=entries_schema)
df_treatments = col_treatments.find_pandas_all({}, schema=treatments_schema)
df_devicestatus = col_devicestatus.find_pandas_all({}, schema=devicestatus_schema)

# Convert the time string to datetime format (not converting time zone this time!)
df_entries["time"] = pd.to_datetime(df_entries["dateString"])


With only a little bit of fussing, I managed to grab the code I wrote previously to plot time in range. I had to adjust it to accept more generic input arguments, but it pretty much works as it's meant to. The only issue is that we have a really large gap in the data between early August and late September (when we started Looping). Not sure what happened there, and not sure those data are recoverable.

In [ ]:
df_cgm_daily = oop.daily_cgm_stats(df_entries["time"], df_entries["sgv"])

sp.daily_tir(df_entries["time"], df_entries["sgv"]);

# Attempt to pull loop settings at specific times

Let's see if I can pull together the things I've been building... I've got CGM data at specific times, and I've functions that return the loop settings at any requested set of times. Seems promising!


In [ ]:
in_times = df_entries["time"]
bg = df_entries["sgv"]

# Pulling settings at each CGM time - making sure it's working!
basal = oop.get_setting_at_times(in_times, col_profile, req_setting = "basal", req_profile = "Default")
carb_ratio = oop.get_setting_at_times(in_times, col_profile, req_setting = "carbratio", req_profile = "Default")
sens = oop.get_setting_at_times(in_times, col_profile, req_setting = "sens", req_profile = "Default")

df_bg = pd.DataFrame(np.transpose([in_times.dt.tz_convert('US/Eastern'), bg, basal, carb_ratio, sens]), 
                                  columns=['time', 'bg', 'basal', 'carb_ratio', 'sens'],
                                  index=in_times.dt.tz_convert('US/Eastern')).dropna()

# Sort the time indices
df_bg.sort_index(inplace=True)

In [ ]:
df_bg.head()

### BG vs time

The next three plots are all just showing the same thing - BG vs time, just looking at it using different plotting methods. First, a plain matplotlib plot. Second, a plot built using hvplot. Third, a built-in hvplot interactive plot where you have some additional GUI controls for exploring the dataframe.

In [ ]:
fig2, ax2 = plt.subplots(1,1)
ax2.plot(in_times, bg, '.');


In [ ]:
import hvplot.pandas #noqa

df_bg.hvplot.scatter(x='time', y='bg')

In [ ]:
explorer = df_bg.hvplot.explorer(x='time', y='bg', kind='scatter', by=['sens'])
explorer

## Splitting out time chunks

I am not really that interested in the earlier data - before that big gap. We didn't have Loop set up during that time period so there's less to look at on that side of things. And the newer data are more relevant to determining how to update treatment settings etc. 

So now I'm going to make a little function that pulls out only the time period I'm intrested in. 

In [ ]:
df_sub = df_bg.loc['2023-09-27':]
df_sub.head()

In [ ]:
df_sub.hvplot.scatter(x='time', y='bg')